## Trabajo Práctico Nº 4
### Referido a  base de datos vectoriales. chromadb.

# Resolución

## Comentarios previos
Dado la gran cantidad de errores que tuve con mi entorno de transformers en mac, opté por usar la librería de open ia para los embedding. El concepto es el mismo, por lo que pude avanzar logrando el resultado esperado.

## Código base

In [60]:
# 1️⃣ Importar librerías
import openai
import chromadb
from chromadb.config import Settings
import os
import uuid

In [61]:
collection_name = "documentos_ia_tp4"
tp_base_path="/Users/fabricio.denardi/Documents/CEIA/BDIA/repos/MIA_01c_BDIA/TP4"

# Configurar la clave de API de OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")  # Usa una variable de entorno para mayor seguridad

# Inicializar ChromaDB con persistencia
client = chromadb.PersistentClient(path="./chroma_store")
collection = client.get_or_create_collection(name=collection_name)

In [62]:
# Función para obtener embeddings usando OpenAI
def get_embeddings(text):
    try:
        response = openai.embeddings.create(
            input=text,
            model="text-embedding-ada-002"  # Modelo de OpenAI para embeddings
        )
        return response.data[0].embedding
    except openai.error.OpenAIError as e:
        print(f"❌ Error al obtener embeddings: {e}")
        return None

In [63]:
def cargar_documentos(base_path=tp_base_path):
    docs_path = os.path.join(base_path, "docs")
    documentos = []
    metadatos = []
    ids = []

    for filename in os.listdir(docs_path):
        if filename.endswith(".txt"):
            with open(os.path.join(docs_path, filename), 'r', encoding="utf-8") as f:
                text = f.read().strip()
                chunks = [text[i:i+500] for i in range(0, len(text), 500)]
                
                for i, chunk in enumerate(chunks):
                    documentos.append(chunk)
                    metadatos.append({"source": filename})
                    ids.append(str(uuid.uuid4()))  # ID único
    
    return documentos, metadatos, ids

In [64]:
def crear_base_datos():
    docs, metas, ids = cargar_documentos()
    embeddings = [get_embeddings(d) for d in docs if get_embeddings(d) is not None]
    collection.add(documents=docs, embeddings=embeddings, metadatas=metas, ids=ids)
    print("✅ Base de datos creada con éxito.")

In [65]:
def create_example(new_doc):
    """Añadir un documento nuevo"""
    embedding = get_embeddings(new_doc)
    if embedding:
        doc_id = str(uuid.uuid4())
        metadata = {"source": "nuevo_documento.txt"}
        
        collection.add(
            documents=[new_doc],
            embeddings=[embedding],
            metadatas=[metadata],
            ids=[doc_id]
        )
        print(f"✅ Documento añadido con ID: {doc_id}")
    else:
        print("❌ No se pudo añadir el documento debido a un error en los embeddings.")

In [66]:
def read_example(query, n_results = 1):
    """Consultar documentos similares"""
    query_emb = get_embeddings(query)
    if query_emb:
        results = collection.query(
            query_embeddings=[query_emb],
            n_results=n_results
        )
        
        if results["documents"]:
            for i in range(n_results):
                print(f"📄 Mejor coincidencia nro {i}: {results['documents'][i-1][0]}")
                print(f"📁 Fuente: {results['metadatas'][i-1][0]['source']}")
                print(f"🆔 ID: {results['ids'][i-1][0]}")
        else:
            print("⚠️ No se encontraron resultados.")
    else:
        print("❌ No se pudo realizar la consulta debido a un error en los embeddings.")

In [67]:
def get_documents_by_id(doc_id):
    """Obtener un documento por ID"""
    results = collection.get(ids=[doc_id], include=["documents", "metadatas", "embeddings"])
    
    if results["documents"]:
        print(f"📄 Documento: {results['documents'][0]}")
        print(f"📁 Fuente: {results['metadatas'][0]['source']}")
    else:
        print("⚠️ Documento no encontrado.")

In [68]:
def update_example(old_id, new_text):
    """Actualizar un documento (eliminar y reinsertar)"""
    collection.delete(ids=[old_id])
    
    new_id = str(uuid.uuid4())
    embedding = get_embeddings(new_text)
    if embedding:
        metadata = {"source": "documento_actualizado.txt"}
        
        collection.add(
            documents=[new_text],
            embeddings=[embedding],
            metadatas=[metadata],
            ids=[new_id]
        )
        print(f"♻️ Documento actualizado. Nuevo ID: {new_id}")
    else:
        print("❌ No se pudo actualizar el documento debido a un error en los embeddings.")

In [69]:
def delete_example(doc_id):
    """Eliminar por ID"""
    collection.delete(ids=[doc_id])
    print(f"🗑️ Documento con ID '{doc_id}' eliminado.")

print('listo')

listo


In [70]:
crear_base_datos()

Delete of nonexisting embedding ID: 16cf4733-7d2a-431d-83a6-fa931a194a15
Delete of nonexisting embedding ID: 16cf4733-7d2a-431d-83a6-fa931a194a15
Delete of nonexisting embedding ID: no_existe
Delete of nonexisting embedding ID: aa24e152-bf34-4a71-b144-a159063035e3
Delete of nonexisting embedding ID: b0398ae5-4ded-43ec-91bb-a118d4b6d144
Delete of nonexisting embedding ID: b0398ae5-4ded-43ec-91bb-a118d4b6d144


✅ Base de datos creada con éxito.


In [71]:
get_embeddings("El sol se alzaba sobre las colinas de Buenos Aires, pintando el cielo con tonos naranja y rosa.")

[0.012318962253630161,
 -0.009295104071497917,
 0.002148367464542389,
 -0.014790205284953117,
 -0.009282685816287994,
 0.02247713878750801,
 -0.0173980500549078,
 0.00825196597725153,
 0.010350660420954227,
 -0.03395165503025055,
 0.001121528446674347,
 0.0060104611329734325,
 -0.0008133992087095976,
 -0.006525821052491665,
 -0.010934320278465748,
 -0.014591512270271778,
 0.010021574795246124,
 -0.011226151138544083,
 0.021943150088191032,
 -0.0075875865295529366,
 -0.008934972807765007,
 0.009425495751202106,
 0.00026039639487862587,
 0.01660327799618244,
 -0.00861209724098444,
 0.0035516361240297556,
 0.019310468807816505,
 -0.02134707197546959,
 0.02943139150738716,
 -0.00740131177008152,
 0.028810475021600723,
 -0.008270593360066414,
 -0.010083666071295738,
 -0.017149683088064194,
 -0.02306079864501953,
 -0.022266026586294174,
 -0.0026994298677891493,
 -0.01220098789781332,
 -0.01713726483285427,
 0.005175330210477114,
 0.006175003945827484,
 -0.003256701398640871,
 0.0045109502971

In [72]:
results = collection.get(include=["embeddings", "documents", "metadatas"])

In [73]:
for id, content, metadata, embedding in zip(results["ids"], results["documents"], results["metadatas"], results["embeddings"]):
    print(f"ID: {id}, Contenido: {content[:100]} ", f"Metadata: {metadata}", f"Embedding: {embedding[:5]}...")

ID: 600a8800-8398-49f6-8566-42704da9ac68, Contenido: El sol se alzaba sobre las colinas de Buenos Aires, pintando el cielo con tonos naranja y rosa. El a  Metadata: {'source': 'doc1.txt'} Embedding: [0.002886376576498151, 0.0004124291008338332, -0.019652780145406723, -0.027064379304647446, 0.002653928007930517]...
ID: b0378bf6-3412-4a95-bf42-b43a3e4fffa6, Contenido: ciosa continuaba, con el tráfico constante y el murmullo de las conversaciones. Los turistas se mara  Metadata: {'source': 'doc1.txt'} Embedding: [-0.011876767501235008, -0.010162802413105965, 0.017784904688596725, -0.010875274427235126, 0.008072437718510628]...
ID: 93f2c684-4888-42d9-9580-44fd3a847dd4, Contenido:  los restaurantes servían deliciosas parrilladas.

Hacia el norte, el elegante barrio de Recoleta in  Metadata: {'source': 'doc1.txt'} Embedding: [-0.004808980040252209, 6.465915066655725e-05, 0.01111509557813406, -0.008399512618780136, 0.013895489275455475]...
ID: 0e37e936-fec0-4c65-9f34-2f829bc61ae6, Contenido: 

## Resolución de las preguntas

### Ejercicio 1: Crear y Consultar 
 
Crea un documento nuevo con el texto: "La gravedad es una fuerza fundamental en el 
universo". 
Realiza una consulta para encontrar documentos similares a la frase: "Fuerzas naturales 
que gobiernan el cosmos". 
 
¿Qué tipo de operación CRUD usaste primero? 
¿Cuál es el ID del documento creado? (Usa collection.peek()). 
Explica por qué el documento aparece como coincidencia en la consulta.

In [74]:
# Crear un documento nuevo
nuevo_texto = "La gravedad es una fuerza fundamental en el universo."
create_example(nuevo_texto)

# Consultar los documentos similares
consulta = "Fuerzas naturales que gobiernan el cosmos."
read_example(consulta)

# Verificar el ID del documento creado usando lo solicitado
count = collection.count()
peek_result = collection.peek(count)
print(f'ID del documento creado: {peek_result["ids"][-1]}')



✅ Documento añadido con ID: 0c868f1e-502c-47c5-8d79-cefa96a636cd
📄 Mejor coincidencia nro 0: La gravedad es una fuerza fundamental en el universo.
📁 Fuente: nuevo_documento.txt
🆔 ID: 0c868f1e-502c-47c5-8d79-cefa96a636cd
ID del documento creado: 0c868f1e-502c-47c5-8d79-cefa96a636cd


#### Explicación
El documento aparece como coincidencia porque el modelo de embeddings identifica similitudes semánticas entre el texto del documento y la consulta.

### Ejercicio 2: Actualizar y Eliminar 
 
Crea un documento con el texto: "Redes neuronales imitan procesos cerebrales". 
Encuentra su ID, luego actualízalo por: "Deep Learning es un subcampo de IA con redes 
neuronales profundas". 
Elimina el documento original usando el mismo ID. 
 
¿Qué ocurre si intentas actualizar un documento eliminado? 
¿Cómo se garantiza la integridad de los datos en las operaciones?

In [75]:
# Crear un documento nuevo
nuevo_doc = "Redes neuronales imitan procesos cerebrales"
create_example(nuevo_doc)

# Encontrar su ID
count = collection.count()
peek_result = collection.peek(count)
doc_id = peek_result['ids'][-1]  # Asumimos que el último documento añadido es el nuevo
print(f"ID del documento creado: {doc_id}")

# Actualizar el documento
nuevo_texto = "Deep Learning es un subcampo de IA con redes neuronales profundas"
update_example(doc_id, nuevo_texto)

# Eliminar el documento original usando el mismo ID
delete_example(doc_id)

# Intentar actualizar un documento eliminado
try:
    update_example(doc_id, "Intento de actualización tras eliminación")
except Exception as e:
    print(f"Error al intentar actualizar un documento eliminado: {e}")

✅ Documento añadido con ID: 5b5636bf-661d-4ef0-a038-7fdb2e08ef57
ID del documento creado: 5b5636bf-661d-4ef0-a038-7fdb2e08ef57


Delete of nonexisting embedding ID: 5b5636bf-661d-4ef0-a038-7fdb2e08ef57
Delete of nonexisting embedding ID: 5b5636bf-661d-4ef0-a038-7fdb2e08ef57
Delete of nonexisting embedding ID: 5b5636bf-661d-4ef0-a038-7fdb2e08ef57
Delete of nonexisting embedding ID: 5b5636bf-661d-4ef0-a038-7fdb2e08ef57


♻️ Documento actualizado. Nuevo ID: d21e8c19-fd81-416a-b08a-6ace4e1cc914
🗑️ Documento con ID '5b5636bf-661d-4ef0-a038-7fdb2e08ef57' eliminado.
♻️ Documento actualizado. Nuevo ID: bbb3001d-3c58-400f-8c21-88ce931f64ca


#### Explicación sobre la integridad de los datos
La integridad de los datos se garantiza mediante el uso de IDs únicos para cada documento. 
Las operaciones como actualización y eliminación de estos verifican la existencia del documento antes de proceder, asegurando que no se realicen cambios en documentos inexistentes.

### Ejercicio 3: Errores y Validación 
Intenta eliminar un documento con el ID "no_existe". 
Crea un nuevo documento sin usar get_embeddings() (ejemplo: enviar un array vacío). 
 
¿Qué errores ocurren en cada caso? 
¿Cómo prevenir estos problemas?

In [76]:
# Intentar eliminar un documento con un ID inexistente
try:
    collection.delete(ids=["no_existe"])
    print("🗑️ Documento eliminado.")
except Exception as e:
    print(f"Error al intentar eliminar un documento inexistente: {e}")

# Intentar crear un documento sin usar get_embeddings()
try:
    collection.add(
        documents=["Este es un documento sin embeddings."],
        embeddings=[],  # Array vacío
        metadatas=[{"source": "sin_embeddings.txt"}],
        ids=[str(uuid.uuid4())]
    )
    print("Documento creado sin embeddings.")
except Exception as e:
    print(f"Error al intentar crear un documento sin embeddings: {e}")

Delete of nonexisting embedding ID: no_existe
Delete of nonexisting embedding ID: no_existe


🗑️ Documento eliminado.
Error al intentar crear un documento sin embeddings: list index out of range


#### Prevención de problemas
Prevención de problemas
1. Verificar la existencia del ID antes de intentar eliminarlo.
2. Asegurarse de generar embeddings válidos antes de añadir documentos.
3. Implementar validaciones en el código para manejar errores de forma adecuada.

### Ejercicio 4: Consultas por Tema 
 Crea tres documentos: 
"Python es ideal para ciencia de datos" 
"JavaScript maneja interacciones web dinámicas" 
"Java se usa en aplicaciones empresariales" 
Realiza una consulta por la frase: "Lenguajes backend y frontend". 
 
¿Cuáles documentos aparecen como coincidencias? 
Analiza cómo las palabras clave influyen en los resultados.

In [77]:
# Crear los tres documentos
create_example("Python es ideal para ciencia de datos")
create_example("JavaScript maneja interacciones web dinámicas")
create_example("Java se usa en aplicaciones empresariales")

# Realizar la consulta
consulta = "Lenguajes backend y frontend"
read_example(consulta)

✅ Documento añadido con ID: b88f3353-accb-4ad3-85a3-abfd45a49aa0
✅ Documento añadido con ID: 92185703-1eba-4f25-ac65-3f44fda37fdd
✅ Documento añadido con ID: b9cb43c3-61a2-4a34-a8e0-867b19448d0e
📄 Mejor coincidencia nro 0: JavaScript maneja interacciones web dinámicas
📁 Fuente: nuevo_documento.txt
🆔 ID: 92185703-1eba-4f25-ac65-3f44fda37fdd


#### Explicación
Los documentos que aparecen como coincidencias son aquellos cuyos embeddings tienen mayor similitud semántica con la consulta.
Las palabras clave como 'backend', 'frontend', 'interacciones web' o 'aplicaciones empresariales' influyen en los resultados, ya que el modelo de embeddings evalúa la relación semántica entre los términos.

### Ejercicio 5: Eliminaciones Selectivas 
Crea dos documentos con tópicos opuestos (ejemplo: uno sobre "Ecología" y otro sobre "Minería de datos"). 
Elimina el documento sobre ecología usando su ID. 
Verifica que solo quede un documento. 
 
¿Cómo usar collection.count() para confirmar?

In [78]:
# Crear dos documentos con tópicos opuestos
create_example("La ecología estudia las relaciones entre los seres vivos y su entorno.")
create_example("La minería de datos permite extraer patrones útiles de grandes volúmenes de información.")

# Obtener los IDs de los documentos creados
count = collection.count()
peek_result = collection.peek(count)
doc_id_ecologia = peek_result['ids'][-2]  # Asumimos que el penúltimo documento es el de ecología
doc_id_mineria = peek_result['ids'][-1]  # Asumimos que el último documento es el de minería de datos

# Eliminar el documento sobre ecología usando su ID
delete_example(doc_id_ecologia)

# Verificar que solo quede un documento
remaining_count = collection.count()
print(f"Número de documentos restantes en la colección: {remaining_count}")

✅ Documento añadido con ID: d9d8f814-065e-4978-a670-2ddd615d7035
✅ Documento añadido con ID: 704c6799-fa78-4f40-b782-a5a41ba5b6e5
🗑️ Documento con ID 'd9d8f814-065e-4978-a670-2ddd615d7035' eliminado.
Número de documentos restantes en la colección: 79


### Ejercicio 6: Actualización en Cadena 
 Crea un documento con el texto "IA y ética son temas actuales". 
Actualízalo dos veces: 
Primera actualización: "Ética en IA aborda dilemas morales" 
Segunda actualización: "El uso de IA debe estar regulado por principios éticos". 
 
¿Cómo sigue evolucionando el contenido del documento con cada update? 

In [79]:
# Crear un documento con el texto inicial
texto_inicial = "IA y ética son temas actuales"
create_example(texto_inicial)

# Obtener el ID del documento recién creado
peek_result = collection.peek()
doc_id_ia_etica = peek_result['ids'][-1]  # Asumimos que el último documento añadido es el nuevo
print(f"ID del documento creado: {doc_id_ia_etica}")

# Primera actualización
texto_actualizado_1 = "Ética en IA aborda dilemas morales"
update_example(doc_id_ia_etica, texto_actualizado_1)

# Segunda actualización
texto_actualizado_2 = "El uso de IA debe estar regulado por principios éticos"
update_example(doc_id_ia_etica, texto_actualizado_2)

✅ Documento añadido con ID: 65ca7346-5a69-44c4-8449-7dc27a9f98c8
ID del documento creado: ace5813a-267e-4256-8f0b-451b41b83a43


Delete of nonexisting embedding ID: ace5813a-267e-4256-8f0b-451b41b83a43
Delete of nonexisting embedding ID: ace5813a-267e-4256-8f0b-451b41b83a43


♻️ Documento actualizado. Nuevo ID: 0f3da92a-a892-41d5-be30-0fe5a0572f48
♻️ Documento actualizado. Nuevo ID: 91175da4-ce52-4973-ae90-b832182d5c7f


#### Explicación
El contenido del documento evoluciona con cada actualización, reflejando los cambios realizados. 
Cada vez que se actualiza, el documento original se elimina y se reemplaza con una nueva versión, manteniendo un nuevo ID único para garantizar la integridad de los datos.

### Ejercicio 7: Consultas Ambiguas 
 
Crea dos documentos similares pero no idénticos: 
"Blockchain garantiza transacciones seguras" 
"Criptomonedas usan tecnología blockchain". 
Realiza una consulta por "Seguridad en finanzas digitales". 
 
¿Ambos documentos aparecen? Explica la similitud semántica. 

In [80]:
# Crear dos documentos similares pero no idénticos
create_example("Blockchain garantiza transacciones seguras")
create_example("Criptomonedas usan tecnología blockchain")

# Realizar la consulta
consulta = "Seguridad en finanzas digitales"
read_example(consulta)
read_example(consulta,n_results=2)


✅ Documento añadido con ID: b5f3689e-e25f-4978-92b2-0a14d8756e1b
✅ Documento añadido con ID: f533c5a4-e9ff-40d0-9606-169d3c1ea46b
📄 Mejor coincidencia nro 0: Blockchain garantiza transacciones seguras
📁 Fuente: nuevo_documento.txt
🆔 ID: b5f3689e-e25f-4978-92b2-0a14d8756e1b
📄 Mejor coincidencia nro 0: Blockchain garantiza transacciones seguras
📁 Fuente: nuevo_documento.txt
🆔 ID: b5f3689e-e25f-4978-92b2-0a14d8756e1b
📄 Mejor coincidencia nro 1: Blockchain garantiza transacciones seguras
📁 Fuente: nuevo_documento.txt
🆔 ID: b5f3689e-e25f-4978-92b2-0a14d8756e1b


#### Explicación
Ambos documentos pueden aparecer como coincidencias si el modelo de embeddings identifica una similitud semántica entre los términos 'blockchain', 'transacciones seguras', 'criptomonedas' y 'finanzas digitales'.

La similitud semántica se basa en cómo el modelo de embeddings representa las palabras y frases en un espacio vectorial, evaluando la proximidad entre los vectores generados.

### Ejercicio 8: Reiniciar Base de Datos 
 
Elimina todos los documentos usando delete() con el parámetro correcto. 
Vuelve a cargar la base desde cero con crear_base_datos(). 
 
¿Cuántos documentos hay ahora? (Usa collection.count()).

In [81]:
# Eliminar todos los documentos de la colección
collection.delete()

# Volver a cargar la base desde cero
crear_base_datos()

# Contar cuántos documentos hay ahora
document_count = collection.count()
print(f"Número de documentos en la colección después de recargar: {document_count}")

✅ Base de datos creada con éxito.
Número de documentos en la colección después de recargar: 33


### Ejercicio 9: Errores de Inserción 
 
Intenta crear un documento sin texto (ejemplo: cadena vacía). 
Corrige el error usando la función get_embeddings(). 
 
¿Qué lección aprendemos sobre los requisitos de ChromaDB?

In [82]:
# Intentar crear un documento con texto vacío
try:
    create_example("")
except Exception as e:
    print(f"Error al intentar crear un documento con texto vacío: {e}")

✅ Documento añadido con ID: 8eae87e5-d48d-44a3-9a63-7873fd0bced2


No dio error. No hay necesidad de corregir.

### Ejercicio opcional: Retrieval-Augmented Generation 
 
Intente levantar un llm en un endpoint local con ollama o LMStudio. Utilice el llm para 
generar respuestas aumentadas con contexto similar a una consulta dada. 


In [83]:
while True:
    opcion = input("Elige una opción o escribe ayuda: ")
    
    if opcion=="ayuda":
        print("\n--- Menú CRUD ---")
        print("1. Crear documento")
        print("2. Consultar por tema")
        print("3. Actualizar documento (por ID)")
        print("4. Eliminar documento (por ID)")
        print("5. Buscar documento (por ID)")
        print("6. Salir")

    if opcion == "1":
        nuevo_doc = input("Ingresa el texto a añadir: ")
        create_example(nuevo_doc)
        
    elif opcion == "2":
        consulta = input("Consulta: ")
        read_example(consulta)
        
    elif opcion == "3":
        doc_id = input("ID del documento a actualizar: ")
        nuevo_texto = input("Nuevo contenido: ")
        update_example(doc_id, nuevo_texto)
        
    elif opcion == "4":
        doc_id = input("ID del documento a eliminar: ")
        delete_example(doc_id)  

    elif opcion == "5":
        doc_id = input("ID del documento a buscar: ")
        get_documents_by_id(doc_id)
        
    elif opcion == "6":
        print("👋 Saliendo del programa.")
        break

👋 Saliendo del programa.


#### Uso de LM  Studio

In [84]:

import requests
import json

LM_STUDIO_URL = "http://127.0.0.1:1234/v1/chat/completions"  

In [85]:
def query_llama(prompt):
    """Envia un prompt a LM Studio y recibe la respuesta."""
    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 256, 
        # Es posible agregar otros parámetros de generación aca (temperature, top_p, etc.)
    }
    try:
        response = requests.post(LM_STUDIO_URL, headers=headers, data=json.dumps(data))
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except requests.exceptions.RequestException as e:
        print(f"Error al conectar con LM Studio: {e}")
        return None
    except (KeyError, json.JSONDecodeError) as e:
        print(f"Error al procesar la respuesta de LM Studio: {e}")
        return None

In [86]:
def responder_pregunta(pregunta):
    """Busca documentos relevantes y luego consulta a Llama."""
    query_emb = get_embeddings(pregunta)
    results = collection.query(
        query_embeddings=[query_emb],
        n_results=5  # Obtener algunos documentos relevantes como contexto
    )

    if results["documents"]:
        contexto = "\n\n".join(results["documents"][0])
        prompt = f"Basándote en la siguiente información:\n\n{contexto}\n\nResponde a la siguiente pregunta: {pregunta}" # Juntar el contexto obtenido y la pregunta
        respuesta = query_llama(prompt)
        return respuesta
    else:
        return "No se encontraron documentos relevantes para responder a tu pregunta."

In [87]:
crear_base_datos() 

✅ Base de datos creada con éxito.


In [88]:

while True:
    pregunta_usuario = input("Pregúntame algo sobre los documentos (o escribe 'salir'): ")
    if pregunta_usuario.lower() == 'salir':
        break

    print(f"🤖 Pregunta: {pregunta_usuario}")
    respuesta_ia = responder_pregunta(pregunta_usuario)
    if respuesta_ia:
        print(f"🤖 Respuesta de la IA: {respuesta_ia}")

🤖 Pregunta: La música folklórica argentina, con sus ritmos y melodías características
🤖 Respuesta de la IA: <think>
Bueno, primero necesito entender la pregunta que me han hecho: "La música folklórica argentina, con sus ritmos y melodías características". Veo que hay una fragmentada de respuesta previa que me ha ayudado a comprender mejor la información proporcionada.

Primero, identifico que se habla sobre la música folklórica argentina. Sé que esta tipología de música es muy importante en many aspectos, incluyendo su importancia cultural y artística. También noto que mencionan instrumentos específicos como la guitarra, el charango y el bombo legüero. Creo que es fundamental destacar estos instrumentos y sus roles dentro de la música argentina.

Luego, me doy cuenta de que la respuesta incomplete terminaba con "con una prosa exquisita". Asumo que se refería a la expresiónartística o la forma en que esta música se traduce en palabras escritas. Es probable que se trate de un estilo de p

#### Conclusión
Como se ve en el cell output, al pasarle contexto, y darle un fragmento de un documento "La música folklórica argentina, con sus ritmos y melodías características" (ver línea 17 del [doc1.txt](docs/doc1.txt)) el LLM me devuelve una respuesta usando parte de este. Con lo cuál vemos que:
1. Los embeedding están funcionando correctamente, tanto el armado como la búsqueda.
2. El modelo utiliza correctamente el contexto.